# Ideal gas equation of state using grand canonical ensemble transition-matrix Monte Carlo

In this example, the ideal gas equation of state is obtained as a test of the flat histogram method.

In [1]:
params={"cubic_box_length": 8, "beta": 1./1.2, "mu": -3}
script="""
MonteCarlo
Configuration cubic_box_length {cubic_box_length} particle_type /feasst/forcefield/atom.fstprt
Potential Model IdealGas
ThermoParams beta {beta} chemical_potential {mu}
FlatHistogram Macrostate MacrostateNumParticles width 1 min 0 max 50 \
              Bias TransitionMatrix min_sweeps 100
TrialTransfer particle_type 0
CriteriaUpdater trials_per 1e5
CriteriaWriter trials_per 1e5 file_name id_fh.txt
Run until_criteria_complete true
""".format(**params)

def run(script):
    with open('script.txt', 'w') as file: file.write(script)
    import subprocess
    syscode = subprocess.call("~/feasst/build/bin/fst < script.txt > script.log", shell=True, executable='/bin/bash')
    with open('script.log', 'r') as file: print(file.read(), '\n', 'exit:', syscode)
run(script)

# FEASST version: v0.20.0-4-g08d5a7de4e-hwh/pyfeasst
Configuration cubic_box_length 8 particle_type /home/hwh/feasst/forcefield/atom.fstprt  
Potential Model IdealGas  
ThermoParams beta 0.8333333333333334 chemical_potential -3  
FlatHistogram Bias TransitionMatrix Macrostate MacrostateNumParticles max 50 min 0 min_sweeps 100 width 1  
TrialTransfer particle_type 0  
CriteriaUpdater trials_per 1e5  
CriteriaWriter file_name id_fh.txt trials_per 1e5  
Run until_criteria_complete true  
# initializing random number generator with seed: 1655833271
 
 exit: 0


Check the ideal gas relationship, $\beta P = \rho$

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

fh=pd.read_csv('id_fh.txt', comment="#")
#print(fh)
print('N betaPV difference')
for delta_conjugate in np.arange(-5, 5, 0.1):
    fh['ln_prob_rw'] = fh['ln_prob'] + fh['state']*delta_conjugate - fh['ln_prob'].min()  # avoid negative log
    fh['ln_prob_rw'] -= np.log(sum(np.exp(fh['ln_prob_rw'])))   # renormalize
    if fh['ln_prob_rw'].values[-1] < -6:
        #plt.plot(fh['ln_prob_rw'])
        N = (np.exp(fh["ln_prob_rw"]) * fh["state"]).sum()
        betaPV = -fh["ln_prob_rw"][0] - np.log(np.exp(fh["ln_prob_rw"]).sum())
        print(N, betaPV, N-betaPV)
        assert(np.abs(1 - betaPV/N) < 1e-2)

N betaPV difference
0.28260651938960046 0.28243688472004025 0.00016963466956021467
0.3123640140080462 0.312160569899379 0.00020344410866718343
0.3452576004050282 0.3450141889097207 0.0002434114953074884
0.3816180498224627 0.3813276134222697 0.0002904364001930504
0.4218109655460796 0.421465505072794 0.00034546047328559704
0.46624040381102455 0.4658309768551357 0.0004094269558888586
0.5153528518422282 0.5148696342203101 0.00048321762191816386
0.5696415928809494 0.5690740330727739 0.0005675598081754663
0.6296514900217144 0.6289885949347819 0.0006628950869325134
0.6959842234510876 0.6952150228614556 0.0007692005896319243
0.7693040199728768 0.7684182653461628 0.0008857546267140304
0.8503439206649607 0.8493330796640493 0.0010108410009114221
0.9399126438055039 0.9387712512061481 0.0011413925993557505
1.0389021180881615 1.0376295318984148 0.0012725861897466828
1.1482957884091773 1.1468983695744732 0.0013974188347041228
1.269177836159279 1.2676715122664641 0.0015063238928147982
1.40274351042692

Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!